# corpora

> Functions to work with multiple corpora and download and build sample corpora.

In [ ]:
#| default_exp corpora

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import re
import os
from great_tables import GT
import polars as pl
import msgspec
from fastcore.script import call_parse

In [ ]:
#| export
from conc.core import CorpusMetadata, logger
from conc.corpus import Corpus

In [ ]:
#| hide
source_path = f'{os.environ.get("HOME")}/data/'
save_path = f'{os.environ.get("HOME")}/data/conc-test-corpora/'

In [ ]:
#| hide
polars_conf = pl.Config.set_tbl_width_chars(300)
polars_conf = pl.Config.set_fmt_str_lengths(300)

## List available corpora

In [ ]:
#| export
def list_corpora(
		path: str # path to load corpus
		) -> pl.DataFrame: # Dataframe with path, corpus, corpus name, document count, token count
	""" Scan a directory for available corpora """
	
	available_corpora = {'corpus': [], 'name': [], 'date_created': [], 'document_count': [], 'token_count': []}
	for dir in os.listdir(path):
		if os.path.isdir(os.path.join(path, dir)) and os.path.isfile( os.path.join(path, dir, 'corpus.json')):
			with open(os.path.join(path, dir, 'corpus.json'), 'rb') as f:
				data = msgspec.json.decode(f.read(), type=CorpusMetadata)

			available_corpora['corpus'].append(dir)
			for k in ['name', 'document_count', 'token_count', 'date_created']:
				attr = getattr(data, k)
				if isinstance(attr, int):
					attr = f'{attr:,}'
				available_corpora[k].append(attr)

	return pl.DataFrame(available_corpora)

In [ ]:
print(list_corpora(save_path))

┌──────────────────────────────────────────────┬───────────────────────────────────────┬─────────────────────┬────────────────┬─────────────┐
│ corpus                                       ┆ name                                  ┆ date_created        ┆ document_count ┆ token_count │
╞══════════════════════════════════════════════╪═══════════════════════════════════════╪═════════════════════╪════════════════╪═════════════╡
│ introduce-yourself.corpus                    ┆ Introduce Yourself                    ┆ 2025-06-10 09:16:41 ┆ 28             ┆ 9,913       │
│ gutenberg.corpus                             ┆ Gutenberg Corpus                      ┆ 2025-06-09 12:44:36 ┆ 18             ┆ 2,546,286   │
│ us-congressional-speeches-subset-200k.corpus ┆ US Congressional Speeches Subset 200k ┆ 2025-06-09 15:04:47 ┆ 200,000        ┆ 39,963,039  │
│ us-congressional-speeches-subset-100k.corpus ┆ US Congressional Speeches Subset 100k ┆ 2025-06-09 15:03:44 ┆ 100,000        ┆ 19,927,241  │
│ gard

## Get data sources

In [ ]:
#| exporti

toy_data = []
toy_data.append(['1.txt', 'The cat sat on the mat.', 'feline', 'cat'])
toy_data.append(['2.txt', 'The dog sat on the mat.', 'canine', 'dog'])
toy_data.append(['3.txt', 'The cat is meowing.', 'feline', 'cat'])
toy_data.append(['4.txt', 'The dog is barking.', 'canine', 'dog'])
toy_data.append(['5.txt', 'The cat is climbing a tree.', 'feline', 'cat'])
toy_data.append(['6.txt', 'The dog is digging a hole.', 'canine', 'dog'])

In [ ]:
#| hide
# checking on counts above
toy_data_test = [doc[1] for doc in toy_data] 
toy_data_test = [re.findall(r'\b\w+\b|[^\w\s]', text) for text in toy_data_test]
toy_data_test = [token.lower() for sublist in toy_data_test for token in sublist if token.strip()]
#print(toy_data_test)
# token count
print(len(toy_data_test)) # should be 38
# word token count
print(len(toy_data_test) - sum([1 for token in toy_data_test if token == '.'])) # should be 32
toy_data_test_unique = set(toy_data_test)
# unique tokens
print(len(toy_data_test_unique))
toy_data_test_unique_word = set([token for token in toy_data_test_unique if token != '.'])
# unique word tokens
print(len(toy_data_test_unique_word))

# based on this - toy corpus should have ... 
# document_count = 6
# token_count = 38
# word_token_count = 32
# unique_tokens = 15
# unique_word_tokens = 14

38
32
15
14


In [ ]:
#| export
def create_toy_corpus_sources(source_path:str # path to location of sources for building corpora
							 ):
	""" Create txt files and csv to test build of toy corpus. """

	toy_path = os.path.join(source_path, 'toy')
	if not os.path.exists(toy_path):
		os.makedirs(toy_path, exist_ok=True)
	for row in toy_data:
		with open(f'{source_path}/toy/{row[0]}', 'w', encoding='utf-8') as f:
			f.write(row[1])
	df = pl.DataFrame(toy_data, orient='row', schema=(('source', str), ('text', str), ('category', str), ('species', str)))
	df.write_csv(os.path.join(source_path, 'toy.csv'))
	df.write_csv(os.path.join(source_path, 'toy.csv.gz'))


In [ ]:
#| hide
#| eval: false
create_toy_corpus_sources(source_path)

In [ ]:
#| export
def show_toy_corpus(
        csv_path:str # path to location of csv for building corpora
        ) -> GT: 
    """ Show toy corpus in a table. """
    
    toy_corpus_df = pl.read_csv(csv_path)
    GT(toy_corpus_df).tab_options(table_margin_left = 0).show()

In [ ]:
show_toy_corpus(os.path.join(source_path, 'toy.csv'))

source,text,category,species
1.txt,The cat sat on the mat.,feline,cat
2.txt,The dog sat on the mat.,canine,dog
3.txt,The cat is meowing.,feline,cat
4.txt,The dog is barking.,canine,dog
5.txt,The cat is climbing a tree.,feline,cat
6.txt,The dog is digging a hole.,canine,dog


In [ ]:
#| export
def get_nltk_corpus_sources(source_path:str # path to location of sources for building corpora
							 ):
	""" Get NLTK corpora as sources for development or testing Conc functionality. """

	try:
		import nltk
	except ImportError as e:
		raise ImportError('This function requires NLTK. To minimise requirements this is not installed by default. You can install NLTK with "pip install nltk"')

	import nltk
	nltk.download('gutenberg')
	nltk.download('brown')
	nltk.download('reuters')
	from nltk.corpus import gutenberg
	from nltk.corpus import reuters
	from nltk.corpus import brown

	def clean_text(text):
		# to match words/punc that followed by /tags
		pattern = re.compile(r"(\S+)(/[^ ]+)") # match non-space followed by / and non-space
		return pattern.sub(r"\1", text)

	if not os.path.exists(source_path):
		os.makedirs(source_path, exist_ok=True)
	if not os.path.exists(f'{source_path}/brown'):
		os.makedirs(f'{source_path}/brown', exist_ok=True)
	brown_path = os.path.join(source_path, 'brown.csv.gz')
	corpus_data = []
	for fileid in brown.fileids():
		corpus_data.append([fileid, clean_text(brown.raw(fileid))])
		with open(f'{source_path}/brown/{fileid}.txt', 'w', encoding='utf-8') as f:
			f.write(clean_text(brown.raw(fileid)))
	df = pl.DataFrame(corpus_data, orient='row', schema=(('source', str), ('text', str)))
	df.write_csv(brown_path)

	gutenberg_path = os.path.join(source_path, 'gutenberg.csv.gz')
	corpus_data = []
	for fileid in gutenberg.fileids():
		corpus_data.append([fileid, clean_text(gutenberg.raw(fileid))])
	df = pl.DataFrame(corpus_data, orient='row', schema=(('source', str), ('text', str)))
	df.write_csv(gutenberg_path)

	reuters_path = os.path.join(source_path, 'reuters.csv.gz')
	corpus_data = []
	for fileid in reuters.fileids():
		fileid_name = fileid.split('/')[1]
		corpus_data.append([fileid_name, clean_text(reuters.raw(fileid))])
	df = pl.DataFrame(corpus_data, orient='row', schema=(('source', str), ('text', str)))
	df.write_csv(reuters_path)


In [ ]:
#| hide
#| eval: false
get_nltk_corpus_sources(source_path)

[nltk_data] Downloading package gutenberg to /home/geoff/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package brown to /home/geoff/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package reuters to /home/geoff/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


The texts for the Brown corpus from nltk can be used to test Conc functionality. The Reuters and Gutenberg corpora are also prepared by `get_nltk_corpus_sources`. Running the function will download the texts and save the texts as a .csv.gz files with columns: source and text. The Brown Corpus is also saved as .txt files to test the Corpus.build_from_texts method.

In [ ]:
#| export
def get_garden_party(source_path: str #path to location of sources for building corpora
					):
	""" Get corpus of The Garden Party by Katherine Mansfield for development of Conc and testing Conc functionality. """

	path = 'https://github.com/ucdh/scraping-garden-party/raw/master/garden-party-corpus.zip'

	import requests
	try:
		import requests
	except ImportError as e:
		raise ImportError('This function requires the requests library. To minimise requirements this is not installed by default. You can install requests with "pip install requests"')

	r = requests.get(path)
	with open(f'{source_path}/garden-party-corpus.zip', 'wb') as f:
		f.write(r.content)
	# converting to .tar and tar.gz files for testing
	import zipfile
	with zipfile.ZipFile(f'{source_path}/garden-party-corpus.zip', 'r') as z:
		z.extractall(f'{source_path}/garden-party-corpus')
	import shutil # make tar.gz
	shutil.make_archive(f'{source_path}/garden-party-corpus', 'gztar', f'{source_path}/garden-party-corpus')
	shutil.move(f'{source_path}/garden-party-corpus.tar.gz', f'{source_path}/garden-party-corpus.tar.gz')
	shutil.make_archive(f'{source_path}/garden-party-corpus', 'tar', f'{source_path}/garden-party-corpus')
	shutil.move(f'{source_path}/garden-party-corpus.tar', f'{source_path}/garden-party-corpus.tar')
	shutil.rmtree(f'{source_path}/garden-party-corpus')
	

The `get_garden_party` function downloads a zip file of an example corpus based on Katherine Mansfield short stories. This function creates a .tar and a .tar.gz version of the texts for testing Corpus build methods.  

In [ ]:
#| eval: false
get_garden_party(source_path)

## Create large corpora for development and testing

In [ ]:
#| export
def get_large_dataset(source_path: str #path to location of sources for building corpora
                    ):
    """ Get 1m rows of https://huggingface.co/datasets/Eugleo/us-congressional-speeches-subset for testing. """
    df = pl.read_parquet('hf://datasets/Eugleo/us-congressional-speeches-subset/data/train-*.parquet')
    df.sample(1000000).select(['speech_id', 'date', 'speaker', 'chamber', 'state', 'text']).write_csv(f'{source_path}/us-congressional-speeches-subset-1m.csv.gz')
    del df


In [ ]:
#| hide
# get_large_dataset(source_path)

In [ ]:
#| hide
# Define the chunk size
# chunk_size = 100_000  # Adjust based on your memory constraints

# # Lazily load the CSV file
# df = pl.scan_csv(f'{source_path}us-congressional-speeches-subset-1m.csv.gz')

# # Add the new column 'is_empty'
# df = df.with_columns(
#     (pl.col('text').str.strip_chars().eq('')).alias('is_empty')
# )

# # get length of is_empty where True
# count = df.filter(pl.col("is_empty") == True).collect().height
# print(f"Number of empty rows: {count}")

# any empty?
#len(df[df['text'].is_null()])

# get distribution of date (by year), speaker, chamber, state
# dates are in iso format - extract year and summarize
# df = df.with_columns(pl.col('date').str.slice(0, 4).alias('year'))
# df.group_by('year').agg(pl.count('year').alias('count')).sort('year', descending=True).head(10).collect()

# #df.group_by('speaker').agg(pl.count('speaker').alias('count')).sort('count', descending=True).head(20)
# #df.group_by('chamber').agg(pl.count('chamber').alias('count')).sort('count', descending=True).head(20)
# df.group_by('state').agg(pl.count('state').alias('count')).sort('count', descending=True).head(20)

In [ ]:
#| export
def create_large_dataset_sizes(source_path: str, #path to location of sources for building corpora
						sizes: list = [10000, 100000, 200000, 500000] # list of sizes for test data-sets
						):
	""" Create datasets of different sizes from data source retrieved by get_large_dataset for testing. """
	for max_i in sizes:
		max_i_label = int(max_i / 1000)
		df = pl.read_csv(f'{source_path}/us-congressional-speeches-subset-1m.csv.gz')
		df.sample(max_i).write_csv(f'{source_path}/us-congressional-speeches-subset-{max_i_label}k.csv.gz')
		logger.info(f'Creating dataset of {max_i_label}k rows')


In [ ]:
#| hide
# create_large_dataset_sizes(source_path)

## Build sample corpora



In [ ]:
#| export
@call_parse
def build_sample_corpora(
		source_path:str, # path to folder with corpora
		save_path:str, # path to save corpora
		force_rebuild:bool = False # force rebuild of corpora, useful for development and testing
		):
	"""Build all test corpora from source files."""

	corpora = {}
	corpora['toy'] = {'name': 'Toy Corpus', 'slug': 'toy', 'description': 'Toy corpus is a very small dataset for testing and library development. ', 'extension': '.csv.gz'}
	corpora['brown'] = {'name': 'Brown Corpus', 'slug': 'brown', 'description': 'A Standard Corpus of Present-Day Edited American English, for use with Digital Computers. by W. N. Francis and H. Kucera (1964) Department of Linguistics, Brown University Providence, Rhode Island, USA Revised 1971, Revised and Amplified 1979 http://www.hit.uib.no/icame/brown/bcm.html. This version downloaded via NLTK https://www.nltk.org/nltk_data/.', 'extension': '.csv.gz'}
	corpora['reuters'] = {'name': 'Reuters Corpus', 'slug': 'reuters', 'description': 'Reuters corpus (Reuters-21578, Distribution 1.0). "The copyright for the text of newswire articles and Reuters annotations in the Reuters-21578 collection resides with Reuters Ltd. Reuters Ltd. and Carnegie Group, Inc. have agreed to allow the free distribution of this data *for research purposes only*. If you publish results based on this data set, please acknowledge its use, refer to the data set by the name (Reuters-21578, Distribution 1.0), and inform your readers of the current location of the data set." https://kdd.ics.uci.edu/databases/reuters21578/reuters21578.html. This version downloaded via NLTK https://www.nltk.org/nltk_data/.', 'extension': '.csv.gz'}
	corpora['gutenberg'] = {'name': 'Gutenberg Corpus', 'slug': 'gutenberg', 'description': 'Project Gutenberg Selections NLTK Corpus. Source: https://gutenberg.org/. Public domain. This version downloaded via NLTK https://www.nltk.org/nltk_data/.', 'extension': '.csv.gz'}
	corpora['garden-party-corpus'] = {'name': 'Garden Party Corpus', 'slug': 'garden-party', 'description': 'A corpus of short stories from The Garden Party: and Other Stories by Katherine Mansfield. Texts downloaded from Project Gutenberg https://gutenberg.org/ and are in the public domain. The text files contain the short story without the title. https://github.com/ucdh/scraping-garden-party', 'extension': '.zip'}

	if not os.path.exists(source_path):
		os.makedirs(source_path)
	if not os.path.exists(save_path):
		os.makedirs(save_path)

	if not os.path.exists(f'{source_path}toy.csv.gz'):
		create_toy_corpus_sources(source_path)

	if not os.path.exists(f'{source_path}garden-party-corpus.zip'):
		get_garden_party(source_path)

	if not os.path.exists(f'{source_path}brown.csv.gz'):
		get_nltk_corpus_sources(source_path)

	for corpus_name, corpus_details in corpora.items():
		if force_rebuild and os.path.isdir(f'{save_path}{corpus_details["slug"]}.corpus'):
			import shutil
			shutil.rmtree(f'{save_path}{corpus_details["slug"]}.corpus', ignore_errors=True)

		try:
			corpus = Corpus().load(f"{save_path}{corpus_details['slug']}.corpus")
		except FileNotFoundError:

			if 'csv' in corpus_details['extension']:
				corpus = Corpus(name = corpus_details['name'], description = corpus_details['description']).build_from_csv(source_path = f'{source_path}{corpus_name}.csv.gz', text_column='text', metadata_columns=['source'], save_path = save_path)
			else:
				corpus = Corpus(name = corpus_details['name'], description = corpus_details['description']).build_from_files(source_path = f'{source_path}{corpus_name}{corpus_details["extension"]}', save_path = save_path)
		except Exception as e:
			raise e
		del corpus

The `build_sample_corpora` function downloads sources and creates sample corpora for development and testing for releases. These datasets are a good way to get started working with Conc. Sample corpora available are:

* Brown Corpus (via NLTK)  
* Gutenberg Corpus (via NLTK)  
* Reuters Corpus (via NLTK)  
* Garden Party Corpus (Katherine Mansfield short stories)  
* Toy Corpus (6 very short texts for testing only)  

After installing Conc you can invoke this function from the command line to download and build the sample corpora:
```sh
conc_build_sample_corpora path/to/save/sources path/to/save/corpora
```


Note: `build_sample_corpora` was accessible via conc.corpus as `build_test_corpora` up to version 0.1.1. This functionality is only accessible from `conc.corpora` now.  

In [ ]:
#| hide
build_sample_corpora(source_path, save_path, force_rebuild=False)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()